In [ ]:
import pandas as pd
import sklearn as sk
import math 
import nltk
import os
import fnmatch
import random
import numpy as np
from xml.etree import ElementTree
import re
import string
import spacy
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup as bs

In [ ]:
# download the stopwords from NLTK
#nltk.download('all')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def makeTextLower(text):
  result = text.lower() 
  return result 

In [ ]:
def MakeNumberRemove(text): 
  #result = re.sub(r'^\d+\s|\s\d+\s|\s\d+$', '', text)
  result = re.sub(r'[-+]?\d+', '', text) 
  return result 

In [ ]:
def MakeHTMLremove(text):
  '''
  result = re.compile('(<.*?>)')
  result = result.sub(r'', text) 
  '''
  cleancode = re.compile('<code>.*?</code>')
  cleanr = re.compile('<.*?>')
  cleanentity = re.compile('&.*;')
  cleantext = re.sub(cleancode, '', text)
  cleantext = re.sub(cleanr, ' ', cleantext)
  cleantext = re.sub(cleanentity, ' ', cleantext)
  
  return cleantext

In [ ]:
def makeRemoveExtraApace(text):
  result = " ".join(text.split())  
  return result   

In [ ]:
def makeRemoveHyperLink(text):
  result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
  return result

In [ ]:
def makeRemovePunctuation(text):
  
  #print(string.punctuation)
  #translator = str.maketrans('', '', string.punctuation)
  result = text=text.translate((str.maketrans('','',string.punctuation)))
  return result
  
  '''
  puncts = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

  for punct in puncts:
    if punct in text:
      text = text.replace(punct, '')

  return text
  '''
#makeRemovePunctuation("Hi;")

In [ ]:
def makeWordLemmatize(text):
  lemmatizer = WordNetLemmatizer()
  #word_tokens = word_tokenize(text)
  lemmas = [lemmatizer.lemmatize(word,pos='v') for word in text] 
  return lemmas 

In [ ]:
def cleanTextData(text):

  #uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'
  text = text.lower()
  # specific
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can\'t", "can not", text)

  # general
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  text = re.sub(r"\n", "", text)
  #text = re.sub(uri_re, "", text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "can not ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"\'scuse", " excuse ", text)
  text = re.sub(r"\'\n", " ", text)
  text = re.sub(r"\'\xa0", " ", text)
  text = re.sub('\s+', ' ', text)
  text = text.strip(' ')
  return text

In [ ]:
def removeStopWords(text):
  stopwords_english = stopwords.words('english')
  texts_clean = []
  #print(text)
  text = word_tokenize(text)
  for word in text: 
    if word not in stopwords_english:  
        texts_clean.append(word)
  return texts_clean

In [ ]:
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words

In [ ]:
def removeSpacyStopWords(text,spacy_stopwords):
  
  texts_clean = []
  #print(text)
  for word in text: 
    if word not in spacy_stopwords:  
        texts_clean.append(word)
  return texts_clean

In [ ]:
def makeStemming(text):
  ps = PorterStemmer() 
  #words = word_tokenize(text)
  lemmas = [ps.stem(word) for word in text] 
  return lemmas  

In [ ]:
def preprocessText(text):

  '''
  - html remove
  - Transforming abbreviations
  - Removing punctuation
  - Lemmatizing/stemming words ?
  - Removing stop words
  '''
  result=re.sub(r"<a.*</a>", '', text)
  result=re.sub(r"<.+?>",'',result)
  #result = makeTextLower(text)
  result=re.sub(r'[^\x00-\x7F]+','',result)
  result = MakeHTMLremove(result)
  #result = makeRemoveExtraApace(result)
  result = makeTextLower(result)
  
  #result = cleanTextData(result)
  result = MakeNumberRemove(result)
  result = makeRemoveHyperLink(result)
  result = makeRemovePunctuation(result)
  
  result = removeStopWords(result)
  result = removeSpacyStopWords(result,spacy_stopwords)
  result = makeWordLemmatize(result)
  result = makeStemming(result)
  
  
  #result = makeTextLower(result)

  return result  

In [ ]:
train_path = "/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/Training/"
#topic name reading
topic_number =0
topic_list = {}
with open("/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/topics.txt") as fp:
  lines = fp.readlines() 
  for line in lines:
    line = line.strip()
    topic_list[line] = topic_number
    topic_number += 1
    
    if topic_number == 3 :
      break
    
fp.close()
print(topic_list)
topicLength = len(topic_list)
#dom = ElementTree.parse(train_path+xml_file[0])
#topic_list = {"Anime":0,"Arduino":1,"Biology":2,"Chess":3}
#topicLength = len(topic_list)
#print(topicLength)
#each topic data size

# train data and validation data 
topicDataSize = 700
trainDataSize = 500
validationDataSize = 200
 
position_array = []
for i in range(0,topicDataSize):
  position_array.append(i)
#print(position_array)
#random.seed(20)
#random.shuffle(position_array)
#print(position_array)

datastore = {}
validstore = {}

#data collection
for topic in topic_list:
  with open(train_path+topic+".xml",'r',encoding='utf-8') as file:
    content = file.read()
    soup = bs(content)
    rowList = soup.findAll("row")
    
    #train data
    for i in range(0,trainDataSize):
      topicPair = (topic,position_array[i])
      row = rowList[position_array[i]]
      if len(row.get('body')) == 0:
        continue
      word_list = preprocessText(row.get('body'))
      if len(word_list) == 0:
        continue
      datastore[topicPair] = np.array(word_list)
    
    #validation data
    for i in range(trainDataSize,trainDataSize+validationDataSize):
      topicPair = (topic,position_array[i])
      row = rowList[position_array[i]]
      if len(row.get('body')) == 0:
        continue
      word_list = preprocessText(row.get('body'))
      if len(word_list) == 0:
        continue
      validstore[topicPair] = np.array(word_list)
'''
for i in validdataStore:
  print(i)
  print(validdataStore[i])
'''      

{'Coffee': 0, 'Arduino': 1, 'Anime': 2}


'\nfor i in validdataStore:\n  print(i)\n  print(validdataStore[i])\n'

In [ ]:
def computeTF(wordDict, doc,uniqueWord):
  tfDict = {}
  corpusCount = len(doc)

  for word, count in zip(uniqueWord,wordDict):
      tfDict[word] = count/float(corpusCount)
  return tfDict

In [ ]:
def numberDocsForWord(datastore,words):
  wordVal = {}
  value = list(datastore.values())

  for word in words:
    count = sum(word in e for e in value)    
    wordVal[word] = count
  
  return wordVal

def uniqueWordFunc(trainDataStore,testStore):
  uniqueWord = []
  for pair in trainDataStore:
    wordList = trainDataStore[pair]
    for word in wordList:
      if word not in uniqueWord:
        uniqueWord.append(word)
  wordval = numberDocsForWord(datastore,uniqueWord)
  for pair in testStore:
    wordList = testStore[pair]
    for word in wordList:
      if word not in uniqueWord:
        uniqueWord.append(word)
  return uniqueWord,wordval



In [ ]:
def countData(datastore,validstore,uniqueWord):

  countDatastore = {}
  countDatastoreTest = {}
  
  #corpusCount = len(datastore)
  #print(uniqueWord)
  for key in datastore:
    tempArray = list(datastore[key])
    wordDictA = []
    for word in uniqueWord:
      wordDictA.append(tempArray.count(word)/float(len(tempArray)))
    countDatastore[key] = wordDictA
    
  
  for key in validstore:
    tempArray = list(validstore[key])
    wordDictA = []
    for word in uniqueWord:
      wordDictA.append(tempArray.count(word)/float(len(tempArray)))
    countDatastoreTest[key] = wordDictA
  
  return countDatastore,countDatastoreTest

uniqueWord,wordval = uniqueWordFunc(datastore,validstore)    
tfData,tfDataTest = countData(datastore,validstore,uniqueWord)
print(len(uniqueWord))
'''
def countTF(countDatastore,countDatastoreTest):
  tfData = {}
  tfDataTest = {}
  for key in countDatastore:
    val = computeTF(countDatastore[key], datastore[key],uniqueWord)
    tfData[key] = val
  
  for key in countDatastoreTest:
    val = computeTF(countDatastoreTest[key], validstore[key],uniqueWord)
    tfDataTest[key] = val
  
  return tfData,tfDataTest

tfData,tfDataTest = countTF(countDatastore,countDatastoreTest)
'''
#del countDatastore
#del countDatastoreTest
#print(countDatastore)
#print(datastore)
#print(tfData)
#print(tfDataTest)

10810


'\ndef countTF(countDatastore,countDatastoreTest):\n  tfData = {}\n  tfDataTest = {}\n  for key in countDatastore:\n    val = computeTF(countDatastore[key], datastore[key],uniqueWord)\n    tfData[key] = val\n  \n  for key in countDatastoreTest:\n    val = computeTF(countDatastoreTest[key], validstore[key],uniqueWord)\n    tfDataTest[key] = val\n  \n  return tfData,tfDataTest\n\ntfData,tfDataTest = countTF(countDatastore,countDatastoreTest)\n'

In [ ]:
def computeIDF(datastore,validstore,numberofDoc,dictionaryWords,wordval,tfData,tfDataTest):
  
  idfDict = {}
  idfDict = dict.fromkeys(set(dictionaryWords), 0)
  #wordval = numberDocsForWord(datastore,dictionaryWords)
  #print(wordval)
  for key in datastore:
    tempArray = datastore[key]
    temp = []
    for word in dictionaryWords:
      if word in tempArray:
        temp.append(math.log10(numberofDoc / (float(wordval[word]) + 1)))
      else :
        temp.append(0.0)
    tfData[key] = list(np.multiply(temp,tfData[key]))

  for key in validstore:
    tempArray = validstore[key]
    temp = []
    for word in dictionaryWords:
      if word in tempArray and word in wordval:
        temp.append(math.log10(numberofDoc / (float(wordval[word]) + 1)))
      else :
        temp.append(0.0)
    tfDataTest[key] = list(np.multiply(temp,tfDataTest[key]))
      
  return tfData,tfDataTest

numberofDoc = len(datastore)
#print(numberofDoc)

idfData = {}
#wordval = numberDocsForWord(datastore,uniqueWord)

print("doneOne")
tfidfDataArray,tfidfDataTestArray = computeIDF(datastore,validstore,numberofDoc,uniqueWord,wordval,tfData,tfDataTest)
'''
for key in datastore:
  idf = computeIDF(datastore[key],numberofDoc,uniqueWord,wordval)
  idfData[key] = idf

idfDataTest = {}
for key in validstore:
  idf = computeIDF(validstore[key],numberofDoc,uniqueWord,wordval)
  idfDataTest[key] = idf
'''
#print(idfData)
#print(idfDataTest)

doneOne


'\nfor key in datastore:\n  idf = computeIDF(datastore[key],numberofDoc,uniqueWord,wordval)\n  idfData[key] = idf\n\nidfDataTest = {}\nfor key in validstore:\n  idf = computeIDF(validstore[key],numberofDoc,uniqueWord,wordval)\n  idfDataTest[key] = idf\n'

In [ ]:
'''
def computeTFIDF(tfBow, idfs):
  tfidf = {}
  for word, val in tfBow.items():
      tfidf[word] = val*idfs[word]
  return tfidf

tfidfData = {}
tfidfDataTest = {}

for key in idfData:
  tfidfData[key] = computeTFIDF(tfData[key],idfData[key])
for key in idfDataTest:
  tfidfDataTest[key] = computeTFIDF(tfDataTest[key],idfDataTest[key])

del tfData
del tfDataTest
del idfData
del idfDataTest
#print(tfidfData)
#print(tfidfDataTest) 
'''

'\ndef computeTFIDF(tfBow, idfs):\n  tfidf = {}\n  for word, val in tfBow.items():\n      tfidf[word] = val*idfs[word]\n  return tfidf\n\ntfidfData = {}\ntfidfDataTest = {}\n\nfor key in idfData:\n  tfidfData[key] = computeTFIDF(tfData[key],idfData[key])\nfor key in idfDataTest:\n  tfidfDataTest[key] = computeTFIDF(tfDataTest[key],idfDataTest[key])\n\ndel tfData\ndel tfDataTest\ndel idfData\ndel idfDataTest\n#print(tfidfData)\n#print(tfidfDataTest) \n'

In [ ]:
'''
def tfidfToArray(tfidfData,tfidfDataTest):
  tfidfDataArray = {}
  tfidfDataTestArray = {}

  for key in tfidfData:
    tempArray = []
    for val in tfidfData[key]:
      tempArray.append(tfidfData[key][val])
    tfidfDataArray[key] = tempArray
  #print(tfidfDataArray)

  for key in tfidfDataTest:
    tempArray = []
    for val in tfidfDataTest[key]:
      tempArray.append(tfidfDataTest[key][val])
    tfidfDataTestArray[key] = tempArray
  #print(tfidfDataArray)
  return tfidfDataArray,tfidfDataTestArray

#tfidfDataArray,tfidfDataTestArray=tfidfToArray(tfidfData,tfidfDataTest) 
#print(tfidfDataArray)
'''

'\ndef tfidfToArray(tfidfData,tfidfDataTest):\n  tfidfDataArray = {}\n  tfidfDataTestArray = {}\n\n  for key in tfidfData:\n    tempArray = []\n    for val in tfidfData[key]:\n      tempArray.append(tfidfData[key][val])\n    tfidfDataArray[key] = tempArray\n  #print(tfidfDataArray)\n\n  for key in tfidfDataTest:\n    tempArray = []\n    for val in tfidfDataTest[key]:\n      tempArray.append(tfidfDataTest[key][val])\n    tfidfDataTestArray[key] = tempArray\n  #print(tfidfDataArray)\n  return tfidfDataArray,tfidfDataTestArray\n\n#tfidfDataArray,tfidfDataTestArray=tfidfToArray(tfidfData,tfidfDataTest) \n#print(tfidfDataArray)\n'

In [ ]:
list1 = np.array([1,2,3])
list2 = np.array([-2,0,3])

np.dot(list1,list2)

7

In [ ]:
list1 = np.array([1,2])
print(np.array(np.sqrt(np.sum((list1)**2))))

2.23606797749979


In [ ]:
'''
allDistances = []
for keyval in tfidfDataTestArray:
  for key in tfidfDataArray:
    #print(key)
    #list1 = np.array(tfidfDataArray[key])
    #list2 = np.array(tfidfDataTestArray[keyval])
    list1Val = np.sqrt(np.sum(np.square(tfidfDataArray[key])))
    list2Val = np.sqrt(np.sum(np.square(tfidfDataTestArray[keyval])))
    sim = np.dot(tfidfDataArray[key],tfidfDataTestArray[keyval])/(list1Val*list2Val)
    #print(list1)
    #print(list2)
    allDistances.append((key,sim))

  allDistances.sort(key=lambda x: x[1],reverse=True)
#print(allDistances)
'''

'\nallDistances = []\nfor keyval in tfidfDataTestArray:\n  for key in tfidfDataArray:\n    #print(key)\n    #list1 = np.array(tfidfDataArray[key])\n    #list2 = np.array(tfidfDataTestArray[keyval])\n    list1Val = np.sqrt(np.sum(np.square(tfidfDataArray[key])))\n    list2Val = np.sqrt(np.sum(np.square(tfidfDataTestArray[keyval])))\n    sim = np.dot(tfidfDataArray[key],tfidfDataTestArray[keyval])/(list1Val*list2Val)\n    #print(list1)\n    #print(list2)\n    allDistances.append((key,sim))\n\n  allDistances.sort(key=lambda x: x[1],reverse=True)\n#print(allDistances)\n'

In [ ]:
def cosineSimilarity(headerCS,testingBinArrCS,tfidfDataTestArray,keyval):
  
  listVal1 = np.sqrt(np.sum(np.square(testingBinArrCS),axis=1))
  listVal2 = np.sqrt(np.sum(np.square(tfidfDataTestArray[keyval])))
  dotResult = np.dot(testingBinArrCS,tfidfDataTestArray[keyval])
  finalResult = np.divide(dotResult,np.dot(listVal1,listVal2))

  finalResult, headerCS = (list(t) for t in zip(*sorted(zip(finalResult, headerCS),reverse=True)))

  return headerCS

In [ ]:
headerCS = list(tfidfDataArray.keys())
testingBinArrCS = np.array(list(tfidfDataArray.values()))

allBinDisctance = {}
for keyval in tfidfDataTestArray:
  predicted = cosineSimilarity(headerCS,testingBinArrCS,tfidfDataTestArray,keyval)
  allBinDisctance[keyval] = predicted

n_neighbors = [1,3,5]
for neighbor in n_neighbors:
  print("Neighbor :",neighbor)
  totalCount = 0
  correctCount = 0
  for keyval in tfidfDataTestArray:
    predArray = allBinDisctance[keyval]
    voting_count = {}
    for val in predArray[:neighbor]:
      if val[0] in voting_count:
        voting_count[val[0]] += 1
      else :
        voting_count[val[0]] = 1

    prediction = max(voting_count, key=voting_count.get)
    if prediction == keyval[0]:
      correctCount += 1
    totalCount += 1
    
  print("Total Correct Count: ",correctCount," Total Wrong Count: ",totalCount-correctCount," Accuracy: ",(correctCount*100)/(totalCount))
print()

Neighbor : 1
Total Correct Count:  548  Total Wrong Count:  33  Accuracy:  94.32013769363166
Neighbor : 3
Total Correct Count:  555  Total Wrong Count:  26  Accuracy:  95.5249569707401
Neighbor : 5
Total Correct Count:  559  Total Wrong Count:  22  Accuracy:  96.21342512908778



In [ ]:
from sklearn.metrics import pairwise_distances

list1=np.array([[1,2]])
list2 = np.array([[3,3]])

pairwise_distances(list1,list2,metric='hamming')

array([[1.]])

In [ ]:
import numpy as np
from itertools import chain
dictionary = {"raj": [1,2], "striver": [2,2], "vikram": [3,4],"pranto":[2,4]} 
value = list(dictionary.values())
sum(1 in e for e in value)

1

In [ ]:
list1=[1,2]
list2 = [2,3]
(list(np.multiply(list1,list2)))

[2, 6]

In [ ]:
!pip3 install bs4
!pip install spacy

import nltk
import math
import os
import fnmatch
import random
import numpy as np
from xml.etree import ElementTree
import re
import string
import spacy
from scipy.spatial import distance
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup as bs

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words

def preprocessText(text):

  def makeTextLower(text):
    result = text.lower() 
    return result 

  def MakeNumberRemove(text): 
    #result = re.sub(r'^\d+\s|\s\d+\s|\s\d+$', '', text)
    result = re.sub(r'[-+]?\d+', '', text) 
    return result 

  def MakeHTMLremove(text):
    cleancode = re.compile('<code>.*?</code>')
    cleanr = re.compile('<.*?>')
    cleanentity = re.compile('&.*;')
    cleantext = re.sub(cleancode, '', text)
    cleantext = re.sub(cleanr, ' ', cleantext)
    cleantext = re.sub(cleanentity, ' ', cleantext)  
    return cleantext

  def makeRemoveExtraApace(text):
    result = " ".join(text.split())  
    return result

  def makeRemoveHyperLink(text):
    result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return result  

  def makeRemovePunctuation(text):
    result = text=text.translate((str.maketrans('','',string.punctuation)))
    return result
  
  def makeWordLemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word,pos='v') for word in text] 
    return lemmas

  def removeStopWords(text):
    stopwords_english = stopwords.words('english')
    texts_clean = []
    #print(text)
    text = word_tokenize(text)
    for word in text: 
      if word not in stopwords_english:  
          texts_clean.append(word)
    return texts_clean

  def removeSpacyStopWords(text,spacy_stopwords):
    texts_clean = []
    #print(text)
    for word in text: 
      if word not in spacy_stopwords:  
          texts_clean.append(word)
    return texts_clean

  def makeStemming(text):
    ps = PorterStemmer() 
    #words = word_tokenize(text)
    lemmas = [ps.stem(word) for word in text] 
    return lemmas  

  result = text
  result=re.sub(r"<.+?>",'',result)
  result=re.sub(r'[^\x00-\x7F]+','',result)
  result = MakeHTMLremove(result)
  result = makeTextLower(result)
  result = MakeNumberRemove(result)
  result = makeRemoveHyperLink(result)
  result = makeRemovePunctuation(result)
  result = removeStopWords(result)
  result = removeSpacyStopWords(result,spacy_stopwords)
  result = makeWordLemmatize(result)
  result = makeStemming(result)
  return result

def runTFIDF(datastore,validstore,topic_list,topicLength,type):

  def numberDocsForWord(datastore,words):
    wordVal = {}
    value = list(datastore.values())
    for word in words:
      count = sum(word in e for e in value)    
      wordVal[word] = count
    return wordVal

  def uniqueWordFunc(trainDataStore,testStore):
    uniqueWord = []
    for pair in trainDataStore:
      wordList = trainDataStore[pair]
      for word in wordList:
        if word not in uniqueWord:
          uniqueWord.append(word)
    wordval = numberDocsForWord(datastore,uniqueWord)
    for pair in testStore:
      wordList = testStore[pair]
      for word in wordList:
        if word not in uniqueWord:
          uniqueWord.append(word)
    return uniqueWord,wordval

  def countData(datastore,validstore,uniqueWord):
    countDatastore = {}
    countDatastoreTest = {}
    
    for key in datastore:
      tempArray = list(datastore[key])
      wordDictA = []
      for word in uniqueWord:
        wordDictA.append(tempArray.count(word)/float(len(tempArray)))
      countDatastore[key] = wordDictA
      
    for key in validstore:
      tempArray = list(validstore[key])
      wordDictA = []
      for word in uniqueWord:
        wordDictA.append(tempArray.count(word)/float(len(tempArray)))
      countDatastoreTest[key] = wordDictA
    
    return countDatastore,countDatastoreTest

  uniqueWord,wordval = uniqueWordFunc(datastore,validstore)    
  tfData,tfDataTest = countData(datastore,validstore,uniqueWord)
  print(len(uniqueWord))

  def computeIDF(datastore,validstore,numberofDoc,dictionaryWords,wordval,tfData,tfDataTest):
    
    idfDict = {}
    idfDict = dict.fromkeys(set(dictionaryWords), 0)
    
    for key in datastore:
      tempArray = datastore[key]
      temp = []
      for word in dictionaryWords:
        if word in tempArray:
          temp.append(math.log10(numberofDoc / (float(wordval[word]) + 1)))
        else :
          temp.append(0.0)
      tfData[key] = list(np.multiply(temp,tfData[key]))

    for key in validstore:
      tempArray = validstore[key]
      temp = []
      for word in dictionaryWords:
        if word in tempArray and word in wordval:
          temp.append(math.log10(numberofDoc / (float(wordval[word]) + 1)))
        else :
          temp.append(0.0)
      tfDataTest[key] = list(np.multiply(temp,tfDataTest[key]))
        
    return tfData,tfDataTest

  numberofDoc = len(datastore)
  #print(numberofDoc)

  idfData = {}
  tfidfDataArray,tfidfDataTestArray = computeIDF(datastore,validstore,numberofDoc,uniqueWord,wordval,tfData,tfDataTest)
  
  def cosineSimilarity(headerCS,testingBinArrCS,tfidfDataTestArray,keyval):
    listVal1 = np.sqrt(np.sum(np.square(testingBinArrCS),axis=1))
    listVal2 = np.sqrt(np.sum(np.square(tfidfDataTestArray[keyval])))
    dotResult = np.dot(testingBinArrCS,tfidfDataTestArray[keyval])
    finalResult = np.divide(dotResult,np.dot(listVal1,listVal2))
    finalResult, headerCS = (list(t) for t in zip(*sorted(zip(finalResult, headerCS),reverse=True)))
    return headerCS

  headerCS = list(tfidfDataArray.keys())
  testingBinArrCS = np.array(list(tfidfDataArray.values()))

  allBinDisctance = {}
  for keyval in tfidfDataTestArray:
    predicted = cosineSimilarity(headerCS,testingBinArrCS,tfidfDataTestArray,keyval)
    allBinDisctance[keyval] = predicted

  if type is 'val':
    n_neighbors = [1,3,5]
  elif type is 'test':
    n_neighbors = [5]

  for neighbor in n_neighbors:
    print("Neighbor :",neighbor)
    totalCount = 0
    correctCount = 0
    for keyval in tfidfDataTestArray:
      predArray = allBinDisctance[keyval]
      voting_count = {}
      for val in predArray[:neighbor]:
        if val[0] in voting_count:
          voting_count[val[0]] += 1
        else :
          voting_count[val[0]] = 1

      prediction = max(voting_count, key=voting_count.get)
      if prediction == keyval[0]:
        correctCount += 1
      totalCount += 1
      
    print("Total Correct Count: ",correctCount," Total Wrong Count: ",totalCount-correctCount," Accuracy: ",(correctCount*100)/(totalCount))
  print()
  return correctCount/float(totalCount)



train_path = "/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/Training/"
#topic name reading
topic_number =0
topic_list = {}
with open("/content/drive/My Drive/Machine Learning Algorithm/kNN/Data/topics.txt") as fp:
  lines = fp.readlines()
   
  for line in lines:
    line = line.strip()
    topic_list[line] = topic_number
    topic_number += 1
    '''
    if topic_number == 3 :
      break
    '''
fp.close()
print(topic_list)
topicLength = len(topic_list)
# train data and validation data 
topicDataSize = 700
trainDataSize = 500
validationDataSize = 200
allResult = {}
trainDataStore = {}
validdataStore = {}
count = 0

position_array = []
for i in range(0,topicDataSize):
  position_array.append(i)

for topic in topic_list:
  
  with open(train_path+topic+".xml",'r',encoding='utf-8') as file:
    content = file.read()
    soup = bs(content)
    rowList = soup.findAll("row")
    count = 0
    for row in rowList[0:trainDataSize]:
      topicPair = (topic,position_array[count])
      if len(row.get('body')) == 0:
        continue;
      word_list = preprocessText(row.get('body'))
      if len(word_list) == 0:
        continue
      for word in word_list:
        pairWord = (word,topic_list[topic])
        if pairWord in allResult:
          allResult[pairWord] += 1
        else:
          allResult[pairWord] = 1
      count += 1
      trainDataStore[topicPair] = np.array(word_list)

    #validation data
    for i in range(trainDataSize,trainDataSize+validationDataSize):
      topicPair = (topic,position_array[i])
      row = rowList[position_array[i]]
      if len(row.get('body')) == 0:
        continue
      word_list = preprocessText(row.get('body'))
      if len(word_list) == 0:
        continue
      validdataStore[topicPair] = np.array(word_list)

print("Document count :",count)

totalLoop = 50
acc = []
for loop in range(totalLoop):
  testSize = 10
  position_array = []
  for i in range(topicDataSize+loop*testSize,topicDataSize+loop*testSize+testSize):
    position_array.append(i)
  testDatastore = {}

  for topic in topic_list:
    with open(train_path+topic+".xml",'r',encoding='utf-8') as file:
      content = file.read()
      soup = bs(content)
      rowList = soup.findAll("row")
      
      for i in range(len(position_array)):
        topicPair = (topic,position_array[i])
        row = rowList[position_array[i]]
        if len(row.get('body')) == 0:
          continue
        word_list = preprocessText(row.get('body'))
        if len(word_list) == 0:
          continue
        testDatastore[topicPair] = np.array(word_list) 

  accuracy = runTFIDF(trainDataStore,testDatastore,topic_list,topicLength,'test')
  acc.append(accuracy)

print(acc)
print(len(acc))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
{'Coffee': 0, 'Arduino': 1, 'Anime': 2, 'Astronomy': 3, 'Biology': 4, 'Chess': 5, 'Cooking': 6, 'Law': 7, 'Space': 8, 'Windows_Phone': 9, 'Wood_Working': 10}
Document count : 500
19550
Neighbor : 5
Total Correct Count:  97  Total Wrong Count:  11  Accuracy:  89.81481481481481

19537
Neighbor : 5
Total Correct Count:  93  Total Wrong Count:  12  Accuracy:  88.57142857142857

19551
Neighbor : 5
Total Correct Count:  87  Total Wrong Count:  17  Accuracy:  83.65384615384616

19516
Neighbor : 5
Total Correct Count:  80  Total Wrong Count:  24  Accuracy:  76.92307692307692

19687
Neighbor : 5
Total Correct Count:  85  Total Wrong Count:  20  Accuracy:  